<a href="https://colab.research.google.com/github/tmoura/machinelearning/blob/master/Series_Temporais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler

path = 'https://raw.githubusercontent.com/EraylsonGaldino/dataset_time_series/master/goldman.txt'
dados = pd.read_csv(path)

In [ ]:
min = 0
max = 1
scaler = MinMaxScaler(feature_range=(min, max)).fit(dados)
serie_normalizada = scaler.transform(dados.values.reshape(1, -1))

dados['Normalizado'] = serie_normalizada[0]

In [ ]:
def create_sliding_windows(series, window_size):
   
    list_of_sliding_windows = []
    list_size_to_iterate = len(series) - window_size 
    for i in range(0, list_size_to_iterate):
        window = series[i: i + window_size + 1]
        list_of_sliding_windows.append(window)

    return np.array(list_of_sliding_windows).reshape(len(list_of_sliding_windows), window_size+1)

## **DÚVIDAS NA PRÓXIMA CÉLULA**

In [ ]:
qtd_lag = 20

#treinamento e teste
#train_data, test_data = dados['Normalizado'].loc['2000':'2011'], dados['Normalizado'].loc['2012':]
#windows_train = create_sliding_windows(train_data.values, 20) ????????

In [ ]:
def select_lag_acf(serie, max_lag):
    from statsmodels.tsa.stattools import acf
    x = serie[0: max_lag+1]
    
    acf_x, confint = acf(serie, nlags=max_lag, alpha=.05, fft=False,
                             unbiased=False)
    
    
    limiar_superior = confint[:, 1] - acf_x
    limiar_inferior = confint[:, 0] - acf_x

    lags_selecionados = []
    
    for i in range(1, max_lag+1):

        
        if acf_x[i] >= limiar_superior[i] or acf_x[i] <= limiar_inferior[i]:
            lags_selecionados.append(i-1)  #-1 por conta que o lag 1 em python é o 0
    
    #caso nenhum lag seja selecionado, essa atividade de seleção para o gridsearch encontrar a melhor combinação de lags
    if len(lags_selecionados)==0:


        print('NENHUM LAG POR ACF')
        lags_selecionados = [i for i in range(max_lag)]

    print('LAGS', lags_selecionados)

    #inverte o valor dos lags para usar na lista de dados
    lags_selecionados = [max_lag - (i+1) for i in lags_selecionados]

    return lags_selecionados

In [ ]:
lags_sel = select_lag_acf(dados['Normalizado'], 20)

print(lags_sel)